# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#Your code here
df = pd.read_csv("homepage_actions.csv")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
# how many viewers also clicked? 1860

click_ids = set(df[df.action == 'click']['id'].unique())
view_ids = set(df[df.action == 'view']['id'].unique())
print("number of viewers: {}, number of clickers: {} \n".format(len(view_ids), len(click_ids)))
print("number of viewers who didn't click: {} \n".format(len(view_ids - click_ids)))

number of viewers: 6328, number of clickers: 1860 

number of viewers who didn't click: 4468 



In [4]:
# did anyone click who didn't view? No all people who clicked also viewed
print("number of clickers who didn't view: {} \n".format(len(click_ids - view_ids)))

number of clickers who didn't view: 0 



In [5]:
# is there an overlap between control and experiment groups?
ctrl_ids = set(df[df.group == 'control']['id'].unique())
exp_ids = set(df[df.group == 'experiment']['id'].unique())
print("Overlap between control and experiment groups: {}".format(len(ctrl_ids&exp_ids)))

Overlap between control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
#Your code here
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [7]:
# convert clicks to a binary variable on a user by user basis
control = df[df.group == 'control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)
experiment = df[df.group == 'experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)
print("Sample sizes: \t Control: {}, \t Experiment: {}".format(len(control), len(experiment)))
print("Total clicks: \t Control: {}, \t Experiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate: \t Control: {}, \t Experiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes: 	 Control: 3332, 	 Experiment: 2996
Total clicks: 	 Control: 932.0, 	 Experiment: 928.0
Average click rate: 	 Control: 0.2797118847539016, 	 Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [8]:
def p_value_welch_ttest(sample_1, sample_2):
    #import scipy stats
    import scipy.stats as stats
    
    # sample stats
    mu1 = sample_1.mean()
    mu2 = sample_2.mean()
    var1 = sample_1.var()
    var2 = sample_2.var()
    n1 = len(sample_1)
    n2 = len(sample_2)
    
    # calculate t statistic
    num = abs(mu1 - mu2)
    den = np.sqrt((var1/n1) + (var2/n2))
    t_stat = num / den
    print(t_stat)
    
    # calculate degrees of freedom
    num = ((var1/n1) + (var2/n2))**2
    den = (var1**2)/(n1**2 * (n1-1)) + (var2**2)/(n2**2 * (n2-1))
    dof = num/den
    print(dof)
    
    # calculate p_value
    p_val = 1 - stats.t.cdf(t_stat, dof)
    print(p_val)

    return p_val

alpha = 0.05
p_val = p_value_welch_ttest(control.click, experiment.click)
print(p_val < alpha, p_val)

2.615440020788211
6211.149291520622
0.004466402814337078
True 0.004466402814337078


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [13]:
#Your code here
ctrl_click_rate = control.click.mean()
null_exp_click_rate = ctrl_click_rate * len(experiment.click)
null_exp_click_rate

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [10]:
#Your code here
n = len(experiment)
p = ctrl_click_rate
var = n * p * (1 - p)
std = np.sqrt(var)
std

24.568547907005815

In [14]:
z_score = (experiment.click.sum() - null_exp_click_rate) / std
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [15]:
#Your code here
import scipy.stats as stats
p_val = 1 - stats.norm.cdf(z_score)
p_val

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [16]:
# Yes.
# Both of the experiments had a simular outcome

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.